# Uploading Data to Database

Imports

In [273]:
import ast
import os
import re

import pandas as pd
import numpy as np

In [3]:
import pymysql
import pymysql.cursors

## Establish database connection

In [581]:
# Connect to the database
connection = pymysql.connect(host='cis550-2.cmxt8otwhjqc.us-east-2.rds.amazonaws.com',
                             db='cis550',
                             user='cis550',
                             password='cis550eklh',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [328]:
# run this box after you are done with the operations
connection.close()

In [582]:
# test query
with connection.cursor() as cursor:
    # Read a single record
    sql = '''
    SHOW TABLES
    ;
    '''
    cursor.execute(sql)
    result = cursor.fetchall()
    print(result)

[{'Tables_in_cis550': 'artist'}, {'Tables_in_cis550': 'artist_city'}, {'Tables_in_cis550': 'artist_genre'}, {'Tables_in_cis550': 'artist_rank'}, {'Tables_in_cis550': 'city'}]


<hr>

## Create tables in database

In [508]:
# don't run again

table_creation_queries = [
    '''CREATE TABLE IF NOT EXISTS artist (
        artist_id         VARCHAR(25) NOT NULL,
        artist_name       VARCHAR(200) NOT NULL,
        PRIMARY KEY (artist_id)
    );''',
    '''CREATE TABLE IF NOT EXISTS artist_genre (
        artist_id         VARCHAR(25) NOT NULL,
        genre             VARCHAR(50) NOT NULL,
        PRIMARY KEY(artist_id, genre),
        FOREIGN KEY(artist_id) references artist(artist_id)
    );''',
    '''CREATE TABLE IF NOT EXISTS city (
        city_id           INT         NOT NULL,
        city_name         VARCHAR(50) NOT NULL,
        city_state        VARCHAR(50) NOT NULL,
        lat               FLOAT       NOT NULL,
        lon               FLOAT       NOT NULL,
        PRIMARY KEY (city_id)
    );''',
    '''CREATE TABLE IF NOT EXISTS artist_city (
        artist_id         VARCHAR(25) NOT NULL,
        city_id           INT         NOT NULL,
        arr_index         INT         NOT NULL,
        PRIMARY KEY(artist_id, city_id, arr_index),
        FOREIGN KEY(artist_id) references artist(artist_id),
        FOREIGN KEY(city_id) references city(city_id)
    );''',
    '''CREATE TABLE IF NOT EXISTS artist_rank (
        artist_id         VARCHAR(25) NOT NULL,
        year              INT         NOT NULL,
        a_rank            INT         NOT NULL,
        PRIMARY KEY (artist_id, year, a_rank),
        FOREIGN KEY(artist_id) references artist(artist_id)
    );''',
]

with connection.cursor() as cursor:
    for query in table_creation_queries:
        cursor.execute(query)
        result = cursor.fetchall()
        print(result)

connection.commit()

()
()
()
()
()


In [553]:
table_verification_queries = table_creation_queries = [
    '''DESC artist;''',
    '''DESC artist_genre;''',
    '''DESC city;''',
    '''DESC artist_city;''',
    '''DESC artist_rank;'''
]

test = None
with connection.cursor() as cursor:
    
    fields = ['Field', 'Type', 'Null?', 'Key', 'Default', 'Extras']
    print('\t\t'.join(fields))
    print('\t\t'.join(['=' * len(field) for field in fields]))
    print()
    
    tables = ['artist', 'artist_genre', 'city', 'artist_city', 'artist_rank']
    
    for i in range(0, len(table_creation_queries)):
        cursor.execute(table_creation_queries[i])
        result = cursor.fetchall()
        
        print(tables[i] + '\n' + '-' * len(tables[i]))
        for r in result:
            print(*['{:15}'.format(str(x)) for x in r.values()])
        print()

Field		Type		Null?		Key		Default		Extras
=====		====		=====		===		=======		======

artist
------
artist_id       varchar(25)     NO              PRI             None                           
artist_name     varchar(200)    NO                              None                           

artist_genre
------------
artist_id       varchar(25)     NO              PRI             None                           
genre           varchar(50)     NO              PRI             None                           

city
----
city_id         int(11)         NO              PRI             None                           
city_name       varchar(50)     NO                              None                           
city_state      varchar(50)     NO                              None                           
lat             float           NO                              None                           
lon             float           NO                              None                           



In [64]:
# query = ''''''.format(', '.join(tables))

# with connection.cursor() as cursor:
#     cursor.execute(query)
#     result = cursor.fetchall()
#     print(result)

<hr>

## Load data

`artist_rank`

In [215]:
billboard_file = 'clean_billboard.csv'
billboard_df = pd.read_csv(billboard_file)
artist_rank_df = billboard_df
artist_rank_df.head()

,a_rank,year,artist_id
0,1,2018,3TVXtAsR1Inumwj472S9r4
1,9,2018,3TVXtAsR1Inumwj472S9r4
2,69,2018,3TVXtAsR1Inumwj472S9r4
3,1,2018,3TVXtAsR1Inumwj472S9r4
4,9,2018,3TVXtAsR1Inumwj472S9r4


Weird, there definitely shouldn't be the same artist twice for the same rank

In [602]:
print(artist_rank_df.shape)
print(artist_rank_df.drop_duplicates().shape)

(203083, 3)
(203083, 3)


In [603]:
artist_rank_df = artist_rank_df.drop_duplicates()

Before we mark as complete, let's take a closer look. Selecting this artist ID (Drake) to see which 2017 songs he/she had.

In [605]:
artist_rank_df[(artist_rank_df.artist_id == '3TVXtAsR1Inumwj472S9r4') & (artist_rank_df.year == 2017)].sort_values('a_rank')

,a_rank,year,artist_id
118,8,2017,3TVXtAsR1Inumwj472S9r4
119,9,2017,3TVXtAsR1Inumwj472S9r4
98,10,2017,3TVXtAsR1Inumwj472S9r4
155,11,2017,3TVXtAsR1Inumwj472S9r4
152,12,2017,3TVXtAsR1Inumwj472S9r4
149,13,2017,3TVXtAsR1Inumwj472S9r4
120,15,2017,3TVXtAsR1Inumwj472S9r4
146,16,2017,3TVXtAsR1Inumwj472S9r4
70,18,2017,3TVXtAsR1Inumwj472S9r4
66,19,2017,3TVXtAsR1Inumwj472S9r4


Okay, Drake unfortunately did not have 74 out of the top 100 songs at year end of billboard. Going to go back and re-clean the data.

In [676]:
billboard_file = 'billboard_data.csv'
billboard_df = pd.read_csv(billboard_file)
billboard_df.head()

,artist_name,date,rank,year
0,Drake,3/31/18,1,2018
1,Ed Sheeran,3/31/18,2,2018
2,Bruno Mars & Cardi B,3/31/18,3,2018
3,Bebe Rexha & Florida Georgia Line,3/31/18,4,2018
4,Post Malone Featuring Ty Dolla $ign,3/31/18,5,2018


In [677]:
billboard_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307200 entries, 0 to 307199
Data columns (total 4 columns):
artist_name    307200 non-null object
date           307200 non-null object
rank           307200 non-null int64
year           307200 non-null int64
dtypes: int64(2), object(2)
memory usage: 9.4+ MB


Splitting artist name approach: cross-check names in the cleaned artist_df table
- if artist name includes an &
    - check if it matches any of the clean artists with & in their name (e.g. Hall & Oates)
       - if match
           - use that artist's id
       - else
           - split string and look for artists
- else
    - check if it matches any valid artist

In [678]:
ampersand_artists = set([a for a in artist_df.artist_name.tolist() if '&' in a])

In [679]:
def get_artist_id_from_name(name):
    res = artist_df[artist_df.artist_name == name]
    return None if len(res) == 0 else res['artist_id'].values[0]

In [691]:
billboard_regex = re.compile(r'( With )|( X )|( x )|(/)|( & )|( with )|( feat. )|(\))|( featuring )|( Featuring )|( \(featuring)|( Feat. )|( \(Featuring)|(, )')
bad_strings = [' With ', ' & ', ' X ', ' x ', '/', ' with ', ' feat. ', ')', ' featuring ', ' Featuring ', ' (featuring', ' Feat. ', ' (Featuring', ', ']

clean_billboard_objects = []

num_tries = 0
num_misses = 0

for ix, row in billboard_df.iterrows():
    artist_id = None
    if any([x in row['artist_name'] for x in bad_strings]):
        if row['artist_name'] in ampersand_artists:
            artist_id = get_artist_id_from_name(row['artist_name'])
            num_tries += 1
            if artist_id is None:
#                 print('Could not find result for artist \'{}\'!'.format(row['artist_name']))
                num_misses += 1
            else:
                clean_billboard_objects.append({
                    'year': row['year'],
                    'a_rank': row['rank'],
                    'artist_id': artist_id
                })
        else:
            artist_group = [a for a in re.split(billboard_regex, row['artist_name']) if a is not None and a not in bad_strings]
            for a in artist_group:
                artist_id = get_artist_id_from_name(a)
                num_tries += 1
                if artist_id is None:
#                     print('Could not find result for artist \'{}\'!'.format(a))
                    num_misses += 1
                else:
                    clean_billboard_objects.append({
                        'year': row['year'],
                        'a_rank': row['rank'],
                        'artist_id': artist_id
                    })
    else:
        artist_id = get_artist_id_from_name(row['artist_name'])
        num_tries += 1
        if artist_id is None:
#             print('Could not find result for artist \'{}\'!'.format(row['artist_name']))
            num_misses += 1
        else:
            clean_billboard_objects.append({
                'year': row['year'],
                'a_rank': row['rank'],
                'artist_id': artist_id
            })

print('Found {} artists out of {}'.format(num_tries - num_misses, num_tries))

Found 276325 artists out of 360446


In [694]:
artist_rank_df = pd.DataFrame(clean_billboard_objects)
artist_rank_df.head()

,a_rank,artist_id,year
0,1,3TVXtAsR1Inumwj472S9r4,2018
1,2,6eUKZXaKkcviH0Ku9w2n3V,2018
2,3,0du5cEVh5yTK9QJze8zA0C,2018
3,3,4kYSro6naA4h99UJvo89HB,2018
4,4,64M6ah0SkkRsnPGtGiRAbb,2018


In [698]:
artist_rank_df.shape

(276325, 3)

In [696]:
artist_rank_df.drop_duplicates().shape

(209591, 3)

In [699]:
artist_rank_df = artist_rank_df.drop_duplicates()

`artist_genre`

In [119]:
artist_genre_file = 'artists_genres_cleaned.csv'
artist_genre_df = pd.read_csv(artist_genre_file)
artist_genre_df.head()

,artist_id,artist_name,genres
0,7izkpvBGc31hUccjdSNz8k,"Richard ""Groove"" Holmes","['jazz funk', 'jazz organ', 'soul jazz']"
1,1bDWGdIC2hardyt55nlQgG,"""Weird Al"" Yankovic","['comedy rock', 'comic']"
2,1VPmR4DJC1PlOtd0IADAO0,$uicideBoy$,['underground hip hop']
3,0kvp9mzfvoXvGtjSWTgrEb,'In The Heights' Original Broadway Company,"['broadway', 'hollywood']"
4,6Ff53KvcvAj5U7Z1vojB5o,*NSYNC,"['boy band', 'dance pop', 'europop', 'pop', 'p..."


`artist`

In [123]:
artist_df = artist_genre_df[['artist_id', 'artist_name']]
artist_df.head()

,artist_id,artist_name
0,7izkpvBGc31hUccjdSNz8k,"Richard ""Groove"" Holmes"
1,1bDWGdIC2hardyt55nlQgG,"""Weird Al"" Yankovic"
2,1VPmR4DJC1PlOtd0IADAO0,$uicideBoy$
3,0kvp9mzfvoXvGtjSWTgrEb,'In The Heights' Original Broadway Company
4,6Ff53KvcvAj5U7Z1vojB5o,*NSYNC


Gotta flatten the genre column.

In [113]:
flat_artist_generes = []
for ix, artist in artist_genre_df.iterrows():
    for genre in ast.literal_eval(artist['genres']):
        flat_artist_generes.append({
            'artist_id': artist['artist_id'],
            'genre': genre
        })

In [124]:
flat_artist_genre_df = pd.DataFrame(flat_artist_generes)
artist_genre_df = flat_artist_genre_df
artist_genre_df.head()

,artist_id,genre
0,7izkpvBGc31hUccjdSNz8k,jazz funk
1,7izkpvBGc31hUccjdSNz8k,jazz organ
2,7izkpvBGc31hUccjdSNz8k,soul jazz
3,1bDWGdIC2hardyt55nlQgG,comedy rock
4,1bDWGdIC2hardyt55nlQgG,comic


Save the final csv files for upload.

In [731]:
# artist
artist_df.to_csv('final_artist.csv', index=False)
# artist_genre
artist_genre_df.to_csv('final_artist_genre.csv', index=False)
# artist_rank
artist_rank_df.to_csv('final_artist_rank.csv', index=False)

`city`

In [157]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
}

In [176]:
city_df = pd.read_csv('city_states.txt')
city_df = city_df.reset_index().rename(columns={
        'index': 'city_id',
         'City': 'city_name',
        'State': 'city_state', 
     'Latitude': 'lat', 
    'Longitude': 'lon'})
city_df.head()

,city_id,city_name,city_state,lat,lon
0,0,San Juan,PR,18.465901,66.10356
1,1,Bayamon,PR,18.326702,66.17417
2,2,Carolina,PR,18.410462,66.06053
3,3,Amherst,MA,42.367092,72.46457
4,4,Worcester,MA,42.265275,71.87941


In [177]:
# update states to full name
city_df['city_state'] = city_df['city_state'].apply(lambda x: states.get(x))

In [178]:
city_df.head()

,city_id,city_name,city_state,lat,lon
0,0,San Juan,Puerto Rico,18.465901,66.10356
1,1,Bayamon,Puerto Rico,18.326702,66.17417
2,2,Carolina,Puerto Rico,18.410462,66.06053
3,3,Amherst,Massachusetts,42.367092,72.46457
4,4,Worcester,Massachusetts,42.265275,71.87941


In [377]:
city_df = city_df.dropna()

In [730]:
# city
city_df.to_csv('final_city.csv', index=False)

`artist_city`

In [182]:
artist_city_df = pd.read_csv('city_artists_file_cleaned.txt')
artist_city_df = artist_city_df.rename(columns={'Artists': 'artists', 'City': 'city', 'State': 'state'})
artist_city_df.head()

,artists,city,state
0,"['Ray J, Boosie Badazz', 'Ray J, The Mob Group...",Albuquerque,New Mexico
1,"['Shoreline Mafia', 'Shoreline Mafia', 'Shorel...",Anaheim,California
2,"['Sammy Adams', ""Lin-Manuel Miranda, 'In The H...",Ann Arbor,Michigan
3,"['MO3', 'MO3, Moneybagg Yo', 'Mr. Pookie', 'La...",Arlington,Texas
4,"['Ray J, Boosie Badazz', 'Ray J, The Mob Group...",Asheville,North Carolina


In [206]:
def get_artist_id_from_name(name):
    res = artist_df[artist_df.artist_name == name]
    return None if len(res) == 0 else res['artist_id'].values[0]

def get_city_id_from_name(city, state):
    res = city_df[(city_df.city_name == city) & (city_df.city_state == state)]
    return None if len(res) == 0 else res['city_id'].values[0]

flat_city_artist = []

for _, row in artist_city_df.iterrows():

    city_id = get_city_id_from_name(row['city'], row['state'])
    if city_id is None:
        print("Could not find result for {}, {}".format(row['city'], row['state']))

    else:
        for ix, artist_group in enumerate(ast.literal_eval(row['artists'])):
            if ',' in artist_group:
                for artist in artist_group.split(', '):
                    artist_id = get_artist_id_from_name(artist)
                    if artist_id is not None:                    
                        flat_city_artist.append({
                            'city_id': city_id,
                            'artist_id': artist_id,
                            'arr_index': ix
                        })
                    else:
                        print('No artist found for {}'.format(artist))
            else:
                artist_id = get_artist_id_from_name(artist)
                if artist_id is not None:                    
                    flat_city_artist.append({
                        'city_id': city_id,
                        'artist_id': artist_id,
                        'arr_index': ix
                    })
                else:
                    print('No artist found for {}'.format(artist))
#         flat_city_artist.append({
#             'artist_id': artist['artist_id'],
#             'city_id': genre,
#             'arr_index': 
#         })

No artist found for The Mob Group
No artist found for Payso B
No artist found for Payso B
No artist found for Payso B
No artist found for Payso B
No artist found for Payso B
No artist found for Payso B
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Back Talk
No artist found for The Fame Riot
No artist found for The Fame Riot
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for N3WPORT
No artist found for Annaka
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Joseph Duveen
No artist found for LODATO
No artist found for Alec Joseph
No artist found for Chel
No artist found for Chel
No artist found for DJ Spinz
No artist found for K-Major
No artist found for DJ Nan2

No artist found for Navé Monjo
No artist found for Navé Monjo
No artist found for Navé Monjo
No artist found for Navé Monjo
No artist found for Cherub
No artist found for David Ortiz
No artist found for Chikk
No artist found for Chikk
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Dia
No artist found for Dia
No artist found for Dia
No artist found for Louis The Child
No artist found for WYNNE
No artist found for Angela McCluskey
No artist found for Angela McCluskey
No artist found for Angela McCluskey
No artist found for Angela McCluskey
No artist found for Angela McCluskey
No artist found for Angela McCluskey
No artist found for Ange

No artist found for Healing Yoga Meditation Music Consort
No artist found for Chillout Lounge
No artist found for Meditation Awareness
No artist found for Healing Yoga Meditation Music Consort
No artist found for Healing Yoga Meditation Music Consort
No artist found for Chillout Lounge
No artist found for Meditation Awareness
No artist found for Healing Yoga Meditation Music Consort
No artist found for Relajacion Conjunto
No artist found for Musica Reiki
No artist found for Musica Instrumental Para Relajar tus Sentidos
No artist found for Massage Therapy Music
No artist found for Spa
No artist found for Massage Therapy Music
No artist found for Spa
No artist found for Spa
No artist found for Spa
No artist found for Relajacion Conjunto
No artist found for Musica Reiki
No artist found for Musica Instrumental Para Relajar tus Sentidos
No artist found for Relajacion Conjunto
No artist found for Musica Reiki
No artist found for Musica Instrumental Para Relajar tus Sentidos
No artist found f

No artist found for DJ NRD
No artist found for 1017 Eskimo
No artist found for 1017 Eskimo
No artist found for Joe Gifted
No artist found for Fronstreet
No artist found for Fronstreet
No artist found for Dinu-Mihai Stefan
No artist found for Joseph Duveen
No artist found for LODATO
No artist found for LODATO
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for N3WPORT
No artist found for Annaka
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mihai Stefan
No artist found for Dinu-Mi

No artist found for Brendon Urie
No artist found for Brendon Urie
No artist found for Brendon Urie
No artist found for Brendon Urie
No artist found for Brendon Urie
No artist found for Brendon Urie
No artist found for Brendon Urie
No artist found for Brendon Urie
No artist found for Jr.
No artist found for Jr.
No artist found for Jr.
No artist found for Macy Maloy
No artist found for Macy Maloy
No artist found for Ball Greezy
No artist found for DJ NRD
No artist found for Quando Rondo
No artist found for K$upreme
No artist found for 1017 Eskimo
No artist found for 1017 Eskimo
No artist found for Street Money Boochie
No artist found for Street Money Boochie
No artist found for Street Money Boochie
No artist found for Street Money Boochie
No artist found for Melodie Malone
No artist found for Melodie Malone
No artist found for Melodie Malone
No artist found for Melodie Malone
No artist found for Murda Beatz
No artist found for Swae Lee
No artist found for Slim Jxmmi
No artist found for C

...

No artist found for Spiff TV
No artist found for Bishop Cortez Vaughn
No artist found for DJ Tunez
No artist found for Spellz
No artist found for The Dap-Kings Horns
No artist found for Wes Walker
No artist found for Dyl
No artist found for Dyl
No artist found for Dyl
No artist found for Dyl
No artist found for Dyl
No artist found for Mascolo
No artist found for Mascolo
No artist found for Louis The Child
No artist found for Caroline Ailin
No artist found for Caroline Ailin
No artist found for Caroline Ailin
No artist found for Caroline Ailin
No artist found for Axel
No artist found for Axel
No artist found for Axel
No artist found for Axel
No artist found for Axel
No artist found for Axel
No artist found for Axel
No artist found for Axel
No artist found for Rhea Raj
No artist found for Rhea Raj
No artist found for Jamie Ray
No artist found for Matt Wilson
No artist found for Matt Wilson
No artist found for Boogshe
No artist found for Boogshe
No artist found for Boogshe
No artist found

No artist found for Quando Rondo
No artist found for Mary Mary
No artist found for Jeremiah
No artist found for Jeremiah
No artist found for Jeremiah
No artist found for Jeremiah
No artist found for Jeremiah
No artist found for Major Clark Jr.
No artist found for Pokey
No artist found for Sean Curran
No artist found for Big Bank
No artist found for Big Bank
No artist found for Big Bank
No artist found for Big Bank
No artist found for Sherwood Marty
No artist found for DJ Envy
No artist found for Derez Deshon
No artist found for The Family
No artist found for The Family
No artist found for The Family
No artist found for Fiya
No artist found for Bishop Paul S. Morton
No artist found for Sr.
No artist found for Sr.
No artist found for Sr.
No artist found for Sr.
No artist found for Sr.
No artist found for Love Fellowship Choir
No artist found for Erin
No artist found for Erin
No artist found for Erin
No artist found for Erin
No artist found for The Family
No artist found for The Family
No

No artist found for Fredi Walker
No artist found for Hardo
No artist found for Hardo
No artist found for Fredi Walker
No artist found for Wayne Wilcox
No artist found for Aaron Lohr
No artist found for Aaron Lohr
No artist found for Wayne Wilcox
No artist found for Aaron Lohr (Steve)
No artist found for Wayne Wilcox (Gordon)
No artist found for Wayne Wilcox (Gordon)
No artist found for Jesse L. Martin
No artist found for KAYN.
No artist found for Vali Ant
No artist found for Vali Ant
No artist found for Vali Ant
No artist found for Vali Ant
No artist found for Vali Ant
No artist found for Vali Ant
No artist found for Ian Gillan
No artist found for John Gustafson
No artist found for Murray Head
No artist found for Ian Gillan
No artist found for "Jesus Christ Superstar" Apostles
No artist found for Murray Head
No artist found for Alan Doggett
No artist found for Brian Keith
No artist found for Victor Brox
No artist found for Alan Doggett
No artist found for Barry Dennen
No artist found f

No artist found for Namiko
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ulices Chaidez
No artist found for Ariel Camacho
No artist found for Ariel Camacho
No artist found for Ariel Camacho
No artist found for Ariel Camacho
No artist found for Ariel Camacho
No artist found for Louis The Child
No artist found for Ashe
No artist found for Angela McCluskey
No artist found for Bonzai
No artist found for MAX
No artist found for CUT_
No artist found for CUT_
No artist found for CUT_
No artist found for Benny Sings
No artist found for Benny Sings
No artist found for Benny Sings
No artist found for Rome of Sublime with Rome
No artist found for Rome of Sublime with Rome
No artist found for Rome of Sublime with 

[{'arr_index': 0, 'artist_id': '6gbGGM0E8Q1hE511psqxL0', 'city_id': 237},
 {'arr_index': 0, 'artist_id': '6z7xFFHxYkE9t8bwIF0Bvg', 'city_id': 237},
 {'arr_index': 1, 'artist_id': '6gbGGM0E8Q1hE511psqxL0', 'city_id': 237},
 {'arr_index': 7, 'artist_id': '1385hLNbrnbCJGokfH2ac2', 'city_id': 237},
 {'arr_index': 12, 'artist_id': '0Kekt6CKSo0m5mivKcoH51', 'city_id': 237},
 {'arr_index': 14, 'artist_id': '2wOqMjp9TyABvtHdOSOTUS', 'city_id': 237},
 {'arr_index': 15, 'artist_id': '1385hLNbrnbCJGokfH2ac2', 'city_id': 237},
 {'arr_index': 17, 'artist_id': '1385hLNbrnbCJGokfH2ac2', 'city_id': 237},
 {'arr_index': 18, 'artist_id': '2wOqMjp9TyABvtHdOSOTUS', 'city_id': 237},
 {'arr_index': 19, 'artist_id': '0Kekt6CKSo0m5mivKcoH51', 'city_id': 237},
 {'arr_index': 20, 'artist_id': '2wOqMjp9TyABvtHdOSOTUS', 'city_id': 237},
 {'arr_index': 21, 'artist_id': '6MDME20pz9RveH9rEXvrOM', 'city_id': 237},
 {'arr_index': 21, 'artist_id': '1Xylc3o4UrD53lo9CvFvVg', 'city_id': 237},
 {'arr_index': 29, 'artist_id

In [208]:
artist_city_df = pd.DataFrame(flat_city_artist)
artist_city_df.head()

,arr_index,artist_id,city_id
0,0,6gbGGM0E8Q1hE511psqxL0,237
1,0,6z7xFFHxYkE9t8bwIF0Bvg,237
2,1,6gbGGM0E8Q1hE511psqxL0,237
3,7,1385hLNbrnbCJGokfH2ac2,237
4,12,0Kekt6CKSo0m5mivKcoH51,237


In [729]:
# artist_city
artist_city_df.to_csv('final_artist_city.csv', index=False)

<hr>

# Upload to Amazon RDS database

We have the following dataframes:
- artist_df
- artist_genre_df
- artist_rank_df
- city_df
- artist_city_df

Reload the DataFrames from the files.

In [511]:
artist_df = pd.read_csv('final_artist.csv')
artist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7286 entries, 0 to 7285
Data columns (total 2 columns):
artist_id      7286 non-null object
artist_name    7286 non-null object
dtypes: object(2)
memory usage: 113.9+ KB


In [512]:
artist_genre_df = pd.read_csv('final_artist_genre.csv')
artist_genre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32487 entries, 0 to 32486
Data columns (total 2 columns):
artist_id    32487 non-null object
genre        32487 non-null object
dtypes: object(2)
memory usage: 507.7+ KB


In [513]:
artist_rank_df = pd.read_csv('final_artist_rank.csv')
artist_rank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267318 entries, 0 to 267317
Data columns (total 3 columns):
a_rank       267318 non-null int64
year         267318 non-null int64
artist_id    267318 non-null object
dtypes: int64(2), object(1)
memory usage: 6.1+ MB


In [514]:
city_df = pd.read_csv('final_city.csv')
city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
city_id       329 non-null int64
city_name     329 non-null object
city_state    329 non-null object
lat           329 non-null float64
lon           329 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 12.9+ KB


In [515]:
artist_city_df = pd.read_csv('final_artist_city.csv')
artist_city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27016 entries, 0 to 27015
Data columns (total 3 columns):
arr_index    27016 non-null int64
artist_id    27016 non-null object
city_id      27016 non-null int64
dtypes: int64(2), object(1)
memory usage: 633.3+ KB


Declare some useful variables.

In [516]:
upload_query = '''
INSERT INTO {table} ({fields}) VALUES
    {values}
;
'''

def stringify(val):
    return '\'{}\''.format(val) if isinstance(val, str) else str(val)

`artist`

In [519]:
bad_chars = re.compile(r'(\')|(\")|(\_)|(\‘)|(\’)|(\`)|(\“)|(\”)|(\')')

values = ''

def stringify_artist(val):
    return stringify(re.sub(bad_chars, r'\\\g<0>', val))

for ix, row in artist_df.iterrows():
    values += ('(' + ', '.join(list(map(stringify_artist, row.values))) + '), \n')
print(values[:1000])

('7izkpvBGc31hUccjdSNz8k', 'Richard \"Groove\" Holmes'), 
('1bDWGdIC2hardyt55nlQgG', '\"Weird Al\" Yankovic'), 
('1VPmR4DJC1PlOtd0IADAO0', '$uicideBoy$'), 
('0kvp9mzfvoXvGtjSWTgrEb', '\'In The Heights\' Original Broadway Company'), 
('6Ff53KvcvAj5U7Z1vojB5o', '*NSYNC'), 
('1L0y9srZMyh9XUnYGv37IP', '\'Til Tuesday'), 
('7gkRNHOOt7QfhhXf0rEnmj', '+44'), 
('0REMf7H0VP6DwfZ9MbuWph', '10 Years'), 
('0MBIKH9DjtBkv8O3nS6szj', '10,000 Maniacs'), 
('6i6WlGzQtXtz7GcC5H5st5', '10cc'), 
('7urq0VfqxEYEEiZUkebXT4', '112'), 
('19dL0fni8Il6amPuGzxkmn', '12 Gauge'), 
('7m3fe3erw9iO9gs4AeLSG8', 'Night Terrors of 1927'), 
('17lzZA2AlOHwCwFALHttmp', '2 Chainz'), 
('2029eJThczywCgnjE33s33', '2 In A Room'), 
('7JBMlJmJpOk87BZTdbYSHD', '2 Pistols'), 
('18JD8DVlD1fakDAw7E9LFC', '2 Unlimited'), 
('0OarfYzJPCi3lAVZ4nhTTd', '20 Fingers'), 
('1URnnhqYAYcrqrcwql10ft', '21 Savage'), 
('6Fi8CHfO8WGtu3yO8c2Mc4', '2Cellos'), 
('1FyYqlTR8CuFH7eRGd0tpe', '2gether'), 
('1ZwdS5xdxEREPySFridCfh', '2Pac'), 
('1rlHVdoyfG6pQkz

In [520]:
', '.join(artist_df.columns.tolist())

'artist_id, artist_name'

In [521]:
with connection.cursor() as cursor:
    cursor.execute(upload_query.format(
                        table='artist',
                        fields=', '.join(artist_df.columns.tolist()),
                        values=values[:-3]
                   )
    )
    result = cursor.fetchall()
    print(result)

()


In [522]:
connection.commit()

`artist_genre`

In [531]:
values = ''

for ix, row in artist_genre_df.iterrows():
    values += ('(' + ', '.join(list(map(stringify_artist, row.values))) + '), \n')
print(values[:1000])

('7izkpvBGc31hUccjdSNz8k', 'jazz funk'), 
('7izkpvBGc31hUccjdSNz8k', 'jazz organ'), 
('7izkpvBGc31hUccjdSNz8k', 'soul jazz'), 
('1bDWGdIC2hardyt55nlQgG', 'comedy rock'), 
('1bDWGdIC2hardyt55nlQgG', 'comic'), 
('1VPmR4DJC1PlOtd0IADAO0', 'underground hip hop'), 
('0kvp9mzfvoXvGtjSWTgrEb', 'broadway'), 
('0kvp9mzfvoXvGtjSWTgrEb', 'hollywood'), 
('6Ff53KvcvAj5U7Z1vojB5o', 'boy band'), 
('6Ff53KvcvAj5U7Z1vojB5o', 'dance pop'), 
('6Ff53KvcvAj5U7Z1vojB5o', 'europop'), 
('6Ff53KvcvAj5U7Z1vojB5o', 'pop'), 
('6Ff53KvcvAj5U7Z1vojB5o', 'pop christmas'), 
('1L0y9srZMyh9XUnYGv37IP', 'boston rock'), 
('1L0y9srZMyh9XUnYGv37IP', 'dance rock'), 
('1L0y9srZMyh9XUnYGv37IP', 'new romantic'), 
('1L0y9srZMyh9XUnYGv37IP', 'new wave'), 
('1L0y9srZMyh9XUnYGv37IP', 'new wave pop'), 
('1L0y9srZMyh9XUnYGv37IP', 'synthpop'), 
('7gkRNHOOt7QfhhXf0rEnmj', 'emo'), 
('7gkRNHOOt7QfhhXf0rEnmj', 'pop punk'), 
('0REMf7H0VP6DwfZ9MbuWph', 'alternative metal'), 
('0REMf7H0VP6DwfZ9MbuWph', 'christian alternative rock'), 
('0REM

In [532]:
', '.join(artist_genre_df.columns.tolist())

'artist_id, genre'

In [537]:
with connection.cursor() as cursor:
    cursor.execute(upload_query.format(
                        table='artist_genre',
                        fields=', '.join(artist_genre_df.columns.tolist()),
                        values=values[:-3]
                   )
    )
    result = cursor.fetchall()
    print(result)

()


In [538]:
connection.commit()

`city`

In [523]:
values = ''

for ix, row in city_df.iterrows():
    values += ('(' + ', '.join(list(map(stringify, row.values))) + '), \n')
print(values[:1000])

(0, 'San Juan', 'Puerto Rico', 18.465901, 66.10356), 
(1, 'Bayamon', 'Puerto Rico', 18.326702, 66.17416999999999), 
(2, 'Carolina', 'Puerto Rico', 18.410462, 66.06053), 
(3, 'Amherst', 'Massachusetts', 42.367092, 72.46457), 
(4, 'Worcester', 'Massachusetts', 42.265275, 71.87941), 
(5, 'Boston', 'Massachusetts', 42.370567, 71.02696), 
(6, 'Brighton', 'Massachusetts', 42.349768, 71.10488000000002), 
(7, 'Cambridge', 'Massachusetts', 42.380442, 71.13294), 
(8, 'Somerville', 'Massachusetts', 42.38092, 71.09889), 
(9, 'Waltham', 'Massachusetts', 42.398588, 71.245), 
(10, 'North Attleboro', 'Massachusetts', 41.964376, 71.32644), 
(11, 'Coventry', 'Rhode Island', 41.694251, 71.63678), 
(12, 'Providence', 'Rhode Island', 41.82275, 71.41445), 
(13, 'Bath', 'Maine', 43.867892, 69.82649), 
(14, 'New Haven', 'Connecticut', 41.365709, 72.9275), 
(15, 'Stamford', 'Connecticut', 41.054082, 73.53621), 
(16, 'Clifton', 'New Jersey', 40.877949, 74.14123000000002), 
(17, 'North Bergen', 'New Jersey', 40.

In [524]:
', '.join(city_df.columns.tolist())

'city_id, city_name, city_state, lat, lon'

In [525]:
with connection.cursor() as cursor:
    cursor.execute(upload_query.format(
                        table='city',
                        fields=', '.join(city_df.columns.tolist()),
                        values=values[:-3]
                   )
    )
    result = cursor.fetchall()
    print(result)

()


In [526]:
connection.commit()

`artist_city`

In [539]:
values = ''

for ix, row in artist_city_df.iterrows():
    values += ('(' + ', '.join(list(map(stringify, row.values))) + '), \n')
print(values[:1000])

(0, '6gbGGM0E8Q1hE511psqxL0', 237), 
(0, '6z7xFFHxYkE9t8bwIF0Bvg', 237), 
(1, '6gbGGM0E8Q1hE511psqxL0', 237), 
(7, '1385hLNbrnbCJGokfH2ac2', 237), 
(12, '0Kekt6CKSo0m5mivKcoH51', 237), 
(14, '2wOqMjp9TyABvtHdOSOTUS', 237), 
(15, '1385hLNbrnbCJGokfH2ac2', 237), 
(17, '1385hLNbrnbCJGokfH2ac2', 237), 
(18, '2wOqMjp9TyABvtHdOSOTUS', 237), 
(19, '0Kekt6CKSo0m5mivKcoH51', 237), 
(20, '2wOqMjp9TyABvtHdOSOTUS', 237), 
(21, '6MDME20pz9RveH9rEXvrOM', 237), 
(21, '1Xylc3o4UrD53lo9CvFvVg', 237), 
(29, '0UxV9IbmiHldBsk0dEJD8j', 237), 
(29, '3bxZkzk0PLHcetO9o4oxXn', 237), 
(30, '3bxZkzk0PLHcetO9o4oxXn', 237), 
(35, '1nnKsr9fK7T1Ife3qZiefD', 237), 
(35, '3kc5AFnL1TQQdNaMdSW2UO', 237), 
(36, '6MDME20pz9RveH9rEXvrOM', 237), 
(36, '3Isy6kedDrgPYoTS1dazA9', 237), 
(36, '1zNqDE7qDGCsyzJwohVaoX', 237), 
(40, '7nPbrzSt1apQM9rY5DVqQZ', 237), 
(44, '629Fs7UJp6tWqOYZi8t8ET', 237), 
(50, '5nFt7a5Du2MkdAr1KniXh7', 237), 
(50, '0XKZpHcqhAKTD2mDDCCbo8', 237), 
(51, '0XKZpHcqhAKTD2mDDCCbo8', 237), 
(53, '0tqCXWVRTv

In [541]:
', '.join(artist_city_df.columns.tolist())

'arr_index, artist_id, city_id'

In [542]:
with connection.cursor() as cursor:
    cursor.execute(upload_query.format(
                        table='artist_city',
                        fields=', '.join(artist_city_df.columns.tolist()),
                        values=values[:-3]
                   )
    )
    result = cursor.fetchall()
    print(result)

()


In [543]:
connection.commit()

`artist_rank`

In [701]:
artist_rank_df.shape

(209591, 3)

Getting a broken pipe error, so going to chunk it into 10K chunks

In [702]:
chunk_size = 10000
print('Splitting {:,} rows into {:,} chunks of {:,}\n...\n\n'.format(
    artist_rank_df.shape[0],
    divmod(artist_rank_df.shape[0], chunk_size)[0] + 1,
    chunk_size
))

values_list = []

for i in range(0, artist_rank_df.shape[0], 10000):
    chunk_values = ''
    for ix, row in artist_rank_df[i:i + chunk_size].iterrows():
        chunk_values += ('(' + ', '.join(list(map(stringify, row.values))) + '), \n')
    values_list.append(chunk_values)

print(values_list[0][:1000])

Splitting 209,591 rows into 21 chunks of 10,000
...


(1, '3TVXtAsR1Inumwj472S9r4', 2018), 
(2, '6eUKZXaKkcviH0Ku9w2n3V', 2018), 
(3, '0du5cEVh5yTK9QJze8zA0C', 2018), 
(3, '4kYSro6naA4h99UJvo89HB', 2018), 
(4, '64M6ah0SkkRsnPGtGiRAbb', 2018), 
(4, '3b8QkneNDz4JHKKKlLgYZg', 2018), 
(5, '246dkjvS1zLTtiykXe5h60', 2018), 
(5, '7c0XG5cIJTrrAgEC3ULPiq', 2018), 
(6, '2qxJFvFYMEDqd7ui6kSAcq', 2018), 
(6, '6WY7D3jk8zTrHtmkqqo5GI', 2018), 
(7, '4nDoRrQiYLoBzwC5BhVJzF', 2018), 
(7, '50co4Is1HCEo8bhOyUWKpn', 2018), 
(8, '1Xyo4u8uXC1ZmMpatF05PJ', 2018), 
(8, '2YZyLoL8N0Wb9xBt1NhZWg', 2018), 
(9, '4TEJudQY2pXxVHPE3gD2EU', 2018), 
(9, '3TVXtAsR1Inumwj472S9r4', 2018), 
(10, '2YZyLoL8N0Wb9xBt1NhZWg', 2018), 
(10, '7tYKF4w9nC0nq9CsPZTHyP', 2018), 
(11, '6oMuImdp5ZcFhWP0ESe6mG', 2018), 
(12, '246dkjvS1zLTtiykXe5h60', 2018), 
(12, '1URnnhqYAYcrqrcwql10ft', 2018), 
(14, '6M2wZ9GZgrQXHCFfjv46we', 2018), 
(15, '4DdkRBBYG6Yk9Ka8tdJ9BW', 2018), 
(15, '0iEtIxbK0KxaSlF7G42ZOp', 2018), 
(16, '4nDoRrQiYLoBzwC5BhVJ

In [703]:
', '.join(artist_rank_df.columns.tolist())

'a_rank, artist_id, year'

In [706]:
for i in range(0, len(values_list)):
    with connection.cursor() as cursor:
        cursor.execute(upload_query.format(
                            table='artist_rank',
                            fields=', '.join(artist_rank_df.columns.tolist()),
                            values=values_list[i][:-3]
                       )
        )
        result = cursor.fetchall()
        print(result)

()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()


In [707]:
connection.commit()

Test out that some queries work.

In [708]:
query = '''SHOW TABLES;'''

with connection.cursor() as cursor:
    cursor.execute(query.format(table='artist_city'))
    result = cursor.fetchall()
    for x in result:
        print(x)

{'Tables_in_cis550': 'artist'}
{'Tables_in_cis550': 'artist_city'}
{'Tables_in_cis550': 'artist_genre'}
{'Tables_in_cis550': 'artist_rank'}
{'Tables_in_cis550': 'city'}


In [709]:
query = '''
SELECT
    *
FROM artist a
NATURAL JOIN artist_genre ag
LIMIT 20
;
'''

with connection.cursor() as cursor:
    cursor.execute(query.format(table='artist_city'))
    result = cursor.fetchall()
    for x in result:
        print(x)

{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'chicago soul'}
{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'classic funk rock'}
{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'disco'}
{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'electro'}
{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'funk'}
{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'motown'}
{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'post-disco'}
{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'quiet storm'}
{'artist_id': '001aJOc7CSQVo3XzoLG4DK', 'artist_name': 'One Way', 'genre': 'urban contemporary'}
{'artist_id': '003kZCrZhoxCckVJ2jCiJC', 'artist_name': 'Peter Wolf', 'genre': 'folk christmas'}
{'artist_id': '003kZCrZhoxCckVJ2jCiJC', 'artist_name': 'Peter Wolf', 'genre': 'pub rock'}
{'artist_id':

In [728]:
query = '''
SELECT
    ag.genre AS genre,
    COUNT(*) AS num_songs
FROM artist_rank ar
NATURAL JOIN artist_genre ag
WHERE
    ar.year = 2018
GROUP BY
    genre
ORDER BY 
    num_songs DESC
LIMIT 10
;
'''

with connection.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()
    for x in result:
        print(x)

{'genre': 'pop', 'num_songs': 799}
{'genre': 'rap', 'num_songs': 545}
{'genre': 'trap music', 'num_songs': 400}
{'genre': 'dance pop', 'num_songs': 296}
{'genre': 'pop rap', 'num_songs': 271}
{'genre': 'post-teen pop', 'num_songs': 260}
{'genre': 'southern hip hop', 'num_songs': 234}
{'genre': 'contemporary country', 'num_songs': 186}
{'genre': 'hip hop', 'num_songs': 169}
{'genre': 'country road', 'num_songs': 127}


In [ ]:
query = '''
SELECT
    ag.genre AS genre,
    COUNT(*) AS num_songs
FROM artist_rank ar
NATURAL JOIN artist_genre ag
WHERE
    ar.year = 2018
GROUP BY
    genre
ORDER BY 
    num_songs DESC
LIMIT 10
;
'''

with connection.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()
    for x in result:
        print(x)

In [726]:
connection.commit()